# 0N2_Animate_SatImagery

---
Read in and plot thermal infrared image(s) from MODIS and VIIRS level1b radiances. 

Overlay ERA5 atmospheric reanalysis surface atmospheric conditions (doi: 10.24381/cds.adbb2d47) and SIDEx buoy positions. 

### Import packages

In [1]:
%load_ext autoreload
%autoreload 2

from common_imports import *

from LIB_plot_sat import *
from LIB_plot_VIIRS import *
from LIB_plot_MODIS import *

import matplotlib.path as mpath
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

## Import level1b MODIS files (HDF and GEO)
Download from: https://ladsweb.modaps.eosdis.nasa.gov/

In [2]:
# set paths for location of level1b hdf and geolocation files
# set provided folder type as path to folder and other as []
MainFolder = [] 
SingleFolder = []
#==============================================================
# MainFolder = '/Volumes/Jewell_EasyStore/SIDEx2021/img_buoy_3daily/VIIRSNP_download/'
SingleFolder = '/Users/mackenziejewell/Desktop/temp/VIIRS2/'
# SingleFolder = '/Users/mackenziejewell/Desktop/temp/MODIS/'
#==============================================================


#==============================================================
sensor = 'VIIRS'
# sensor = 'MODIS'
#==============================================================

if str(sensor) == 'VIIRS':
    satellite_labels = [('VNP03MOD','VNP02MOD'), ('VJ103MOD','VJ102MOD')]
elif str(sensor) == 'MODIS':
    satellite_labels = [('MOD03','MOD021KM'), ('MYD03','MYD021KM')]
    

Image_Meta_paired = pair_images_meta(MainFolder = MainFolder, SingleFolder = SingleFolder, 
                                     sensor = sensor, satellite_labels = satellite_labels,
                                     min_geofile_sizeMB = 28, min_imfile_sizeMB = 50, max_diff_minutes = 20)
    

Search in single folder: /Users/mackenziejewell/Desktop/temp/VIIRS2/
Pair 0
------
2021-03-02 14:12:00
2021-03-02 14:18:00

Pair 1
------
2021-03-03 13:54:00
2021-03-03 14:00:00

Pair 2
------
2021-03-03 22:12:00
2021-03-03 22:18:00

Pair 3
------
2021-03-04 21:54:00
2021-03-04 22:00:00

Pair 4
------
2021-03-05 21:36:00
2021-03-05 21:42:00

Pair 5
------
2021-03-07 14:18:00
2021-03-07 14:24:00

Pair 6
------
2021-03-08 14:00:00
2021-03-08 14:06:00

Pair 7
------
2021-03-08 22:18:00
2021-03-08 22:24:00

Pair 8
------
2021-03-09 13:42:00

Pair 9
------
2021-03-09 22:00:00
2021-03-09 22:06:00

Pair 10
------
2021-03-10 21:42:00

Pair 11
------
2021-03-13 14:06:00
2021-03-13 14:12:00

Pair 12
------
2021-03-14 13:48:00

Pair 13
------
2021-03-14 22:06:00
2021-03-14 22:12:00

Pair 14
------
2021-03-15 21:48:00
2021-03-15 21:54:00

Pair 15
------
2021-03-18 14:12:00
2021-03-18 14:18:00

Pair 16
------
2021-03-19 13:54:00

Pair 17
------
2021-03-19 22:12:00
2021-03-19 22:18:00

Pair 18
-----

## Specify which pairs of images to plot

In [3]:
# specify which pairs of images to plot
# either 'All' or index(es) of images to plot (e.g. [1,3,6])
#==============================================================
# RunPair = [0,1,2,3,4,5,6] 
RunPair = np.arange(0, 30) 
# RunPair = 'All'
#==============================================================

## ECMWF data

In [4]:
# name and directory for ECMWF atmospheric data (nc file type)
# or set = None if you don't want to include
#==============================================================
# ECMWF = None
ECMWF = '/Volumes/Jewell_EasyStore/ECMWF/annual/hourly/ERA5_2021.nc'
#==============================================================


## Buoy coordinate data

In [5]:
# set list of path+name of csv files containing coordinates
# or set = None if not adding buoy coordinates
#==============================================================
# csv_directory = None
buoy_file = './data/BuoyCoordinates_cln_v0.nc'
#==============================================================
ds = xr.open_dataset(buoy_file)
ds.close()
buoy_time = pd.to_datetime(ds.time.values)


from LIB_SIDExbuoy import open_buoy_data, calc_velocity, calculate_velocity

def grab_buoy_velocities(ImageDate, ds, buoy_time):
    
    # find index nearest to date
    nearest_ind = np.argmin(np.abs((buoy_time - ImageDate)))
    time_slice = buoy_time[nearest_ind-1:nearest_ind+2]

    # grab all buoy coords and velocities
    all_buoys = ds.buoyID.values
    buoy_lons = np.array([])
    buoy_lats = np.array([])
    buoy_u = np.array([])
    buoy_v = np.array([])
    
    for buoy in all_buoys:
    
        crop_time = ds.sel(time=time_slice).sel(buoyID=buoy)

        # calc velocity
        out = calculate_velocity(lons = crop_time.longitude.values, lats = crop_time.latitude.values, 
                                 times = pd.to_datetime(crop_time.time.values), method = 'centered')
        u, v, sp, time, lat, lon, dx, dy, di, az, dt = out

        if len(u) == 0:
            buoy_lons = np.append(buoy_lons, np.nan)
            buoy_lats = np.append(buoy_lats, np.nan)
            buoy_u = np.append(buoy_u, np.nan)
            buoy_v = np.append(buoy_v, np.nan)

        else:
            buoy_lons = np.append(buoy_lons, lon)
            buoy_lats = np.append(buoy_lats, lat)
            buoy_u = np.append(buoy_u, u.magnitude)
            buoy_v = np.append(buoy_v, v.magnitude)

    return buoy_lons, buoy_lats, buoy_u, buoy_v


## Specify other plot parametres and layers

In [6]:
# PLOT PARAMETERS

# set plot range and map_projection
#==============================================================
lat_range = [69.5, 78]
lon_range = [197, 232]
extent = [lon_range[0], lon_range[1], lat_range[0], lat_range[1]]
map_projection = ccrs.NorthPolarStereo(central_longitude=205)
#==============================================================

# specify band of data to plot
#==============================================================
if str(sensor) == 'MODIS':
    band = '31'   # Thermal MODIS: Infrared (TIR) at 10.780–11.280 micrometers
elif str(sensor) == 'VIIRS':
    band = 'M15'  # Thermal VIIRS: longwave IR 10.26 - 11.26 micrometers
#==============================================================


# set colorscale for image
#==============================================================
ice_cmap = mpl.cm.Blues
# ice_cmap = cmocean.cm.ice_r
#==============================================================


# whether or not to suppress prints
#==============================================================
quiet = True
#==============================================================

# hide known warnings that result in many printed warning statements
#==============================================================
# ignore shapely warning for geographic plots
import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
# pcolormesh shading warning
warnings.filterwarnings("ignore", module = "matplotlib\..*" )
warnings.filterwarnings("ignore", module = "cartopy\..*" )
#==============================================================


In [7]:
def flatten_pair_data(lat, lon, _image_):
    
    print(len(lat))
    
    all_lats = np.array([])
    all_lons = np.array([])
    all_rad = np.array([])

    all_lats = np.append(all_lats, lat[0].flatten())
    all_lons = np.append(all_lons, lon[0].flatten())
    all_rad = np.append(all_rad, _image_[0].flatten())
    
    if len(lat) == 2:
        all_lats = np.append(all_lats, lat[1].flatten())
        all_lons = np.append(all_lons, lon[1].flatten())
        all_rad = np.append(all_rad, _image_[1].flatten())
        
    return all_lats, all_lons, all_rad

## make plot

In [ ]:
starttime = datetime.now()

# RUN TO MAKE PLOT, ADJUST APPEARANCE OF LAYERS BELOW AS NEEDED
#==============================================================

# Run through all pairs of images given in RunPair and make plots
#----------------------------------------------------------------
if str(RunPair) == 'All':
    RunPair = np.arange(0,np.max(Image_Meta_paired[:,4])+1)
for ii in RunPair:
    # grab metadata from current_set of paired images
    #------------------------------------------------
    # grab current_set of paired images to run through
    current_set = Image_Meta_paired[np.where(Image_Meta_paired[:,4]==ii)[0]]
    # start empty lists to fill with image names, paths, and dates
    IMG_filename=[]
    GEO_filename=[]
    # add data from all images in current_set to above lists
    counter = 0
    for image_meta in current_set:
        # grab date and ImageName for saving from first file in current_set
        if counter == 0:
            ImageDate = image_meta[0]
            ImageName = image_meta[3]+image_meta[2][0:22]
        IMG_filename = np.append(IMG_filename, image_meta[3]+image_meta[2])
        GEO_filename = np.append(GEO_filename, image_meta[3]+image_meta[1])
        counter+=1
    

    if not quiet:
        print('Save with name {}'.format(ImageName))    
        print('Image is from: {} UTC (day {} of {})'.format(ImageDate,ImageDate.strftime('%j'), ImageDate.strftime('%Y')))
    # grab data from current_set of paired images
    #--------------------------------------------
    # start empty lists to fill with imagery data and coordinates
    _image_ = []
    lat = []
    lon = []

    # for all images in current_set
    for jj in range(len(current_set)):

        # import imagery data, mask invalid values
        # and import geo data
        #-----------------------------------------

        if str(sensor) == 'MODIS':
            _level1bimage_ = load_MODISband(IMG_filename[jj], 'EV_1KM_Emissive', band, 'radiance')
            LAT, LON = get_MODISgeo(GEO_filename[jj])

        elif str(sensor) == 'VIIRS':
            _level1bimage_ = load_VIIRS_band(IMG_filename[jj], band = 'M15')
            LAT, LON = get_VIIRS_geo(GEO_filename[jj])


        # add imagery and coordinates for this file to the lists
        #-------------------------------------------------------
        _image_.append(_level1bimage_)
        lat.append(LAT)
        lon.append(LON)


    # create custom colorscale
#         _min, _max = colorscale_range(_image_)
#         CSCALE = [_min+0.5, _max-0.5]

    #==============
    # PLOT IMAGERY
    #==============
    # create figure
    #--------------
    # static scale
#         cscale = [3, 5]

    # dynamic color scale
    all_lats, all_lons, all_rad = flatten_pair_data(lat, lon, _image_)
    all_rad[all_rad.data == 65533] = np.nan

#         all_rad = ma.masked_where(all_rad, all_rad == 65533)

#         all_lats = (np.array(lat)).flatten()
#         all_lons = (np.array(lon)).flatten()
#         all_rad = (np.array(_image_)).flatten()
    cond = (all_lats > 70).astype(int)+(all_lats < 77).astype(int)+(all_lons > 200).astype(int)+(all_lons < 240).astype(int)
    min_val = np.nanpercentile(all_rad.data[cond == 4], 1)-0.1
    max_val = np.nanpercentile(all_rad.data[cond == 4], 99)+0.8+1
    cscale = [min_val, max_val]

    sp = 5
    fig, ax = plt.subplots(subplot_kw=dict(projection=map_projection), figsize=(8,8), facecolor='white')
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    ax.add_feature(cartopy.feature.OCEAN, color=cmocean.cm.ice_r(0.3), zorder=0)


    plot_singleband_sat(ax,  lat, lon, _image_, cmap=ice_cmap, cscale=cscale, 
                        shading='nearest', zorder=1)

    #======================
    # ADD CARTOPY FEATURES
    #======================
    add_land(ax,  scale='10m', color=[0.85,0.85,0.85], alpha=1, fill_dateline_gap=True, zorder=3)
    add_coast(ax, scale='10m', color=[0.7,0.7,0.7], linewidth=1, alpha=1, zorder=4)
    add_grid(ax, lats=np.arange(70,90,5), lons=np.arange(100,300,20), linewidth=1, color='gray', alpha=0.3, zorder=10)



    # label date
    #======================
    add_date(fig, ax, ImageDate, date_format='%d %b %Y (%H UTC)',
             boxstyle='round,pad=0.2,rounding_size=0.2',
             method='manual', facecolor='white', edgecolor='None',
             x = 0.55, y = -0.02, textcolor='k', fontsize=18, zorder=10)

    # import era5
    #=============
    # grab nearest date index from ECMWF file
    rounded_date = ImageDate-timedelta(minutes = ImageDate.minute)
    if not quiet:
        print('Add wind data: nearest date ECMWF --> {}'.format(rounded_date))
    ds_era = xr.open_dataset(ECMWF).sel(time=rounded_date)
    ds_era.close()
    u10  = ds_era.u10.values
    v10  = ds_era.v10.values
    msl  = ds_era.msl.values/100
    Lons, Lats = np.meshgrid(ds_era.longitude.values, ds_era.latitude.values)


    # wind vectors
    #======================
    # fix vectors for cartopy plotting (only for plotting!!!)
    add_vectors(fig, ax, Lons, Lats, u10, v10, regrid=[7], color=[0.4,0.4,0.4], scale=150, 
                width=0.003, headwidth=4, headaxislength=3, headlength=5, minshaft=1, minlength=1, 
                linewidth=0, alpha=1, angles='uv', pivot='mid',
                quiv_key=None)


    # sea level pressue
    #======================
    lev = np.arange(980,1060,4)
    ax.contour(Lons, Lats, msl, levels = lev, linewidths = (lev-960)/40,
               colors=[[0.4,0.4,0.4]],
               zorder=10, transform=ccrs.PlateCarree())


    # buoys
    #======================
    buoy_lons, buoy_lats, buoy_u, buoy_v = grab_buoy_velocities(ImageDate, ds, buoy_time)
    buoy_speed = np.sqrt(buoy_u**2+buoy_v**2)
    ax.quiver(buoy_lons, buoy_lats, *fix_cartopy_vectors(buoy_u, buoy_v, buoy_lats),
              scale=300, width=0.0025, headwidth=5, transform=ccrs.PlateCarree(), zorder=8)
    ax.scatter(buoy_lons, buoy_lats, s=7,
               c=buoy_speed, cmap=cmocean.cm.speed,vmin=0, vmax=40,
               edgecolor='k', lw=0.5, transform=ccrs.PlateCarree(), zorder=9)




    # inset map
    #======================
    #======================
    #======================
    
    # create inset plots
    #-------------------
    size = 0.4
    axins = inset_axes(ax, width="100%", height="100%", loc='upper left',
                       bbox_to_anchor=(-0.03,0.6,size,size), bbox_transform=ax.transAxes, 
                       axes_class=cartopy.mpl.geoaxes.GeoAxes, 
                       axes_kwargs=dict(map_projection=map_projection))
    # buoys
    #======================
    local_buoys = ['23', '25', '26', '27', '29', '32', '33', '35', '36', '38', 
           '40', '41', '43','46', '48', '49', '50', '51']
    ds_local = ds.sel(buoyID = local_buoys)
    buoy_lons, buoy_lats, buoy_u, buoy_v = grab_buoy_velocities(ImageDate, ds_local, buoy_time)
    buoy_speed = np.sqrt(buoy_u**2+buoy_v**2)
    center_lat = buoy_lats[0]-0.02
    center_lon = buoy_lons[0]-0.175
    buffer_lon = 1
    buffer_lat = buffer_lon*np.cos(center_lat*np.pi/180)
    extent_mini = [center_lon-buffer_lon, center_lon+buffer_lon,
                   center_lat-buffer_lat, center_lat+buffer_lat]
    axins.set_extent(extent_mini, ccrs.PlateCarree())

    theta = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)
    axins.set_boundary(circle, transform=axins.transAxes)

    # buoys
    #======================
    axins.quiver(buoy_lons, buoy_lats, *fix_cartopy_vectors(buoy_u, buoy_v, buoy_lats),
              scale=100, width=0.005, headwidth=5, transform=ccrs.PlateCarree(), zorder=8)
    axins.scatter(buoy_lons, buoy_lats, s=10,
               c=buoy_speed, cmap=cmocean.cm.speed,vmin=0, vmax=40,
               edgecolor='k', lw=0.5, transform=ccrs.PlateCarree(), zorder=9)
    out = verts * radius + center
    axins.plot(out[:,0], out[:,1], c='k', lw=2, transform=axins.transAxes, zorder=10)

    # imagery
    #========
    for ii in range(0,len(_image_)):
        axins.pcolormesh(lon[ii], lat[ii], _image_[ii],vmin=cscale[0], vmax=cscale[1],
                         cmap=ice_cmap, shading='nearest', zorder=1, transform=ccrs.PlateCarree(), 
                         clip_path=(circle, axins.transAxes))

    fig.savefig(ImageName+'_v3.png',bbox_inches="tight", pad_inches = 0, dpi=300)
    fig.clear()
    plt.close(fig) 

    # remove automatic image border
    ax.spines['geo'].set_linewidth(0)


print('\n\n\n')
print(f'>>> runtime: {datetime.now()-starttime}')


2
2
2
2
2
2
2
2
1
2
1
2
1
2
2
2
1
2
2
2
1
2
1
2
